In [1]:
from time import sleep
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from robotsmeco import Robot as rob
from tasho.OCPGenerator import OCPGenerator
import numpy as np
import casadi as cs
import tasho
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env

In [2]:
robot = rob.Robot("kinova")
link_name = 7
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[0, 1, 0, 0.6], [1, 0, 0, 0.0], [0, 0, -1, 0.25], [0, 0, 0, 1]]
    ))
q0 = [1.0193752249977548, -0.05311582280659044, -2.815452580946695,
    1.3191046402052224, 2.8582660722530533, 1.3988994390898029,1.8226311094569714]
robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
ndof = robot.nq
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

Loading robot params from json ...
Loaded 7-DoF robot: kinova


In [3]:
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)

In [4]:
task_P2P.write_task_graph("P2P_Task.svg")

In [5]:
reg_jacc = task_P2P.constraint_expressions['reg_qdd_'+robot.name].change_weight(0.1)
task_P2P.add_path_constraints(Regularization(task_P2P.variables['qd_'+robot.name], 1))

In [6]:
task_P2P.write_task_graph("P2P_Task2.svg")

In [7]:
horizon_steps = 10
horizon_period = 3.0
OCP_gen = OCPGenerator(task_P2P, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)
OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}}
)
OCP_gen.tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.7, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      529
Number of nonzeros in inequality constraint Jacobian.:      217
Number of nonzeros in Lagrangian Hessian.............:      168

Total number of variables............................:      224
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      164
Total number of inequality constrai

In [8]:
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
_, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)

from tasho import WorldSimulator
import pybullet as p

obj = WorldSimulator.WorldSimulator()

    # Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position, orientation, robot.name)
joint_indices = [0, 1, 2, 3, 4, 5, 6]

obj.resetJointState(robotID, joint_indices, q0)

for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetJointState(
        robotID, joint_indices, qsol[i]
    )

sleep(0.5)
obj.end_simulation()

pybullet build time: May 20 2022 19:44:17


Ending simulation


Tunnel-following task

In [9]:
from tasho.templates.SE3_tunnel import SE3Tunnel

vel_limit = 0.5 #m/s
acc_limit = 2.0 #m/s^2

trans_tunnel_size = 0.02
rot_tunnel_size = 5/3.14159/180

link_name = 7
a_p = 0.2
z_p = 0.1
SE3_path_fun = lambda s : cs.vertcat(
                    cs.horzcat(cs.DM([[0, 1, 0], [1, 0, 0], [0, 0, -1]]), 
                    cs.vertcat(0.6+z_p*cs.sin(s*(4*np.pi)), 0.0+a_p*cs.sin(s*(2*np.pi)), 0.25+a_p*cs.sin(s*(2*np.pi))*cs.cos(s*(2*np.pi)))), 
                    cs.DM([0, 0, 0, 1]).T)
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[-1, 0, 0, 0.6], [0, 1, 0, 0.35], [0, 0, -1, 0.25], [0, 0, 0, 1]]
    ))
q0 = [ 0.42280387,  1.56128753, -2.07387664,  1.1543891,   1.7809308,   2.03112421,
4.02677039]

tunnel_disembodiedEE = SE3Tunnel("contouring", SE3_path_fun, vel_limit, acc_limit, trans_tunnel_size, rot_tunnel_size)
tunnel_disembodiedEE.write_task_graph("tunnel_disEE.svg")

In [10]:
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

# # Using the template to create the P2P task
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)
# Removing the goal pose of P2P because not relevant for tunnel-following
task_P2P.remove_expression(goal_pose)
task_P2P.write_task_graph("P2P_Task3.svg")

In [11]:
tunnel_disembodiedEE.substitute_expression(tunnel_disembodiedEE.variables['SE3_traj_contouring'], task_P2P.expressions["pose_7_kinova"])
tunnel_disembodiedEE.write_task_graph("tunnel_disEE2.svg")

In [12]:
tunnel_task = TaskModel.compose("lemniscate", "tunnel_following", tunnel_disembodiedEE, task_P2P)
tunnel_task.write_task_graph("tunnel_following.svg")

In [13]:
horizon_steps = 30
horizon_period = 3
OCP_gen = OCPGenerator(tunnel_task, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)

OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}} 
)

OCP_gen.tc.solve_ocp()

st = OCP_gen.stage_tasks[0]
# print(OCP_gen.tc.sol_sample(st.expressions['SE3_path_contouring'].x))
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
# print(qsol)
t_grid, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)


This is Ipopt version 3.12.7, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:     1705
Number of nonzeros in inequality constraint Jacobian.:      720
Number of nonzeros in Lagrangian Hessian.............:     1320

Total number of variables............................:      736
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      505
Total number of inequality constraints...............:      720
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:      600
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.5000321e-05 1.00e+00 6.06e-04  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1

In [14]:
obj = WorldSimulator.WorldSimulator()

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position, orientation, robot.name)
joint_indices = [0, 1, 2, 3, 4, 5, 6]

obj.resetJointState(robotID, joint_indices, q0)

for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetJointState(
        robotID, joint_indices, qsol[i]
    )

sleep(0.5)
obj.end_simulation()

Ending simulation


Object picking MPC

In [15]:
# Define initial conditions of the robot
q0_val = [0, -0.523598, 0, 2.51799, 0, -0.523598, -1.5708]
qd0_val = [0] * robot.ndof

################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (7,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (7,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, 7, goal_pose, q_init, 0.001)
task_P2P.write_task_graph("P2P_Task4.svg")

In [16]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_kinova'])
task_P2P.remove_terminal_constraints('rot_con_pose_7_kinova_vs_goal',
                                    'trans_con_pose_7_kinova_vs_goal')
task_P2P.write_task_graph("P2P_Task5.svg")

In [17]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_kinova']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_7_kinova_vs_goal'], 1.0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_7_kinova_vs_goal'], 1), 
                            Regularization(task_P2P.variables['qd_kinova'], 1e-4))

task_P2P.write_task_graph("P2P_Task6.svg")

In [18]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 5,"tol": 1e-3}})
# tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 0,"tol": 1e-3, "linear_solver":"ma27"}}) #use this if you have hsl


################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_kinova'].x
qd = pOCP.stage_tasks[0].variables['qd_kinova'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_kinova'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_kinova'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_kinova'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([0, 1, 0, 0.5]),
    cs.hcat([1, 0, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_kinova", "var":"qd_kinova", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_7_kinova_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

mpc_options = default_mpc_options.get_default_mpc_options()


tc.ocp_solver = "ipopt"
tc.ocp_options = mpc_options["ipopt_hsl"]["options"]
tc.mpc_solver = tc.ocp_solver
tc.mpc_options = tc.ocp_options
tc.set_ocp_solver("ipopt", tc.mpc_options)
import os; os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,846]:

b3Printf: /home/mtplnr/mpc_ws/postech_lec/.venv/lib/python3.8/site-packages/robotsme

sh: 1: export: : bad variable name


ing
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


In [19]:
vars_db = tc.generate_MPC_component("./", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti2_p_2[12:15]-fk_T(opti2_x_1){7}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti2_p_2[12:15]-fk_T(opti2_x_1){7}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [20]:
MPC_component = MPC("kinova_obj_pickup", "./" + tc.name + ".json")

In [21]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
kinovaID = obj.add_robot(position, orientation, "kinova")

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

cube2 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_col.urdf", fixed = True)
environment.add_object(cube2, "cube2")
environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")
cube2ID = environment.get_object_ID("cube2")

p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 3, 4, 5, 6]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")
obj.resetJointState(kinovaID, joint_indices, q0_val)
obj.setController(
    kinovaID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)

In [ ]:
# Execute the MPC loop
q_log = []
q_dot_log = []
predicted_pos_log = []

for i in range(horizon_size * 100):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(kinovaID, joint_indices)
    qd_now = obj.readJointVelocities(kinovaID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_kinova"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_kinova"]["val"] = qd_now

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.3
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos_log.append(predicted_pos.full())
    p.resetBasePositionAndOrientation(cube2ID, predicted_pos.full(), [0.0, 0.0, 0.0, 1.0])
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([0, 1, 0, predicted_pos[0]]),
        cs.hcat([1, 0, 0, predicted_pos[1]]),
        cs.hcat([0, 0, -1, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_kinova"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_kinova"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_kinova"]["val"] * t_mpc).full()
    obj.setController(
        kinovaID, "velocity", joint_indices, targetVelocities=qd_control_sig + qdd_control_sig
    )

    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
    if "termination_criteria_true" in MPC_component.event_output_port:
        break

----------- MPC execution -----------
Predicted position of cube [0.5, 0.15777500346789, 0.41]
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.

----------- MPC execution -----------
Predicted position of cube [0.49999968066805, 0.31677326454043, 0.27249110475156]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 194.00us ( 48.50us) 194.84us ( 48.71us)         4
       nlp_g  |  39.00us (  9.75us)  39.33us (  9.83us)         4
    nlp_grad  | 165.00us (165.00us) 165.85us (165.85us)         1
  nlp_grad_f  | 529.00us (132.25us) 528.93us (132.23us)         4
  nlp_hess_l  |  10.35ms (  3.45ms)   2.35ms (783.32us)         3
   nlp_jac_g  |  64.00us ( 16.00us)  62.75us ( 15.69us)         4
       total  |  13.13ms ( 13.13ms)   5.12ms (  5.12ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999968506725, 0.32652397891008, 0.27249052544209]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 186.00us ( 46.50us) 186.66us ( 46.67us)         4
       nlp_g  |  37.00us (  9.25us)  36.95us (  9.24us)         4
    nlp_grad  | 154.00us (154.00us

----------- MPC execution -----------
Predicted position of cube [1.0039346274684, 0.34714236655459, 0.084997406693809]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   2.92ms (486.67us) 293.38us ( 48.90us)         6
       nlp_g  |  72.00us ( 12.00us)  56.78us (  9.46us)         6
    nlp_grad  | 158.00us (158.00us) 158.17us (158.17us)         1
  nlp_grad_f  |  10.69ms (  1.78ms) 789.86us (131.64us)         6
  nlp_hess_l  |  20.54ms (  4.11ms)   3.98ms (796.27us)         5
   nlp_jac_g  | 170.00us ( 28.33us)  96.77us ( 16.13us)         6
       total  |  71.04ms ( 71.04ms)   8.06ms (  8.06ms)         1
----------- MPC execution -----------
Predicted position of cube [1.0168112617753, 0.3470766626552, 0.084992417747751]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 486.00us ( 60.75us) 375.65us ( 46.96us)         8
       nlp_g  |  69.00us (  8.62us)  68.08us (  8.51us)         8
    nlp_grad  | 127.00us (127.00us)

----------- MPC execution -----------
Predicted position of cube [0.7517179877527, 0.3361733857713, 0.031864854445672]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 294.00us ( 49.00us) 295.32us ( 49.22us)         6
       nlp_g  |  54.00us (  9.00us)  53.56us (  8.93us)         6
    nlp_grad  | 187.00us (187.00us) 188.35us (188.35us)         1
  nlp_grad_f  | 771.00us (128.50us) 772.24us (128.71us)         6
  nlp_hess_l  |  19.02ms (  3.80ms)   3.82ms (763.47us)         5
   nlp_jac_g  |  92.00us ( 15.33us)  93.22us ( 15.54us)         6
       total  |  23.09ms ( 23.09ms)   7.88ms (  7.88ms)         1
----------- MPC execution -----------
Predicted position of cube [0.58482693300912, 0.33047986136483, -0.41881785871253]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   7.90ms (  1.13ms) 345.59us ( 49.37us)         7
       nlp_g  |  64.00us (  9.14us)  62.88us (  8.98us)         7
    nlp_grad  | 161.00us (161.00us

----------- MPC execution -----------
Predicted position of cube [0.72423355890625, -1.0670316779217, 0.085348122325812]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.02ms (113.44us) 443.52us ( 49.28us)         9
       nlp_g  |  79.00us (  8.78us)  78.96us (  8.77us)         9
    nlp_grad  | 160.00us (160.00us) 159.61us (159.61us)         1
  nlp_grad_f  |  22.55ms (  2.51ms)   1.19ms (131.83us)         9
  nlp_hess_l  |  34.27ms (  4.28ms)   6.39ms (798.28us)         8
   nlp_jac_g  | 269.00us ( 29.89us) 131.69us ( 14.63us)         9
       total  |  81.03ms ( 81.03ms)  12.30ms ( 12.30ms)         1
----------- MPC execution -----------
Predicted position of cube [0.723504842417, -1.0857715783394, 0.085118770233676]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 587.00us ( 83.86us) 342.02us ( 48.86us)         7
       nlp_g  | 110.00us ( 15.71us)  61.47us (  8.78us)         7
    nlp_grad  | 157.00us (157.00us

...

----------- MPC execution -----------
Predicted position of cube [11.498543931649, -3.0828818488965, 0.084990144809333]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 186.00us ( 62.00us) 186.27us ( 62.09us)         3
       nlp_g  |  41.00us ( 13.67us)  41.29us ( 13.76us)         3
    nlp_grad  | 179.00us (179.00us) 179.39us (179.39us)         1
  nlp_grad_f  | 522.00us (174.00us) 522.58us (174.19us)         3
  nlp_hess_l  |  10.06ms (  5.03ms)   2.06ms (  1.03ms)         2
   nlp_jac_g  |  61.00us ( 20.33us)  61.76us ( 20.59us)         3
       total  |  12.89ms ( 12.89ms)   4.87ms (  4.87ms)         1
----------- MPC execution -----------
Predicted position of cube [11.514775735125, -3.0824939712653, 0.084990134966829]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 141.00us ( 47.00us) 141.93us ( 47.31us)         3
       nlp_g  |  32.00us ( 10.67us)  31.27us ( 10.42us)         3
    nlp_grad  | 674.00us (674.00us

----------- MPC execution -----------
Predicted position of cube [11.710659894311, -3.0781782363906, 0.084989976283037]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 224.00us ( 74.67us) 191.27us ( 63.76us)         3
       nlp_g  |  41.00us ( 13.67us)  41.60us ( 13.87us)         3
    nlp_grad  | 221.00us (221.00us) 221.00us (221.00us)         1
  nlp_grad_f  | 527.00us (175.67us) 528.50us (176.17us)         3
  nlp_hess_l  |   6.57ms (  3.29ms)   2.07ms (  1.04ms)         2
   nlp_jac_g  |  70.00us ( 23.33us)  62.25us ( 20.75us)         3
       total  |  11.51ms ( 11.51ms)   5.08ms (  5.08ms)         1
----------- MPC execution -----------
Predicted position of cube [11.720211359857, -3.0779712599988, 0.084989977726317]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  16.79ms (  5.60ms) 208.21us ( 69.40us)         3
       nlp_g  |  73.00us ( 24.33us)  45.13us ( 15.04us)         3
    nlp_grad  | 216.00us (216.00us

----------- MPC execution -----------
Predicted position of cube [11.243040200884, -3.0756271284438, 0.084989738161603]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 142.00us ( 47.33us) 141.03us ( 47.01us)         3
       nlp_g  |  31.00us ( 10.33us)  30.29us ( 10.10us)         3
    nlp_grad  | 165.00us (165.00us) 165.80us (165.80us)         1
  nlp_grad_f  | 384.00us (128.00us) 385.69us (128.56us)         3
  nlp_hess_l  |   1.53ms (766.50us)   1.53ms (766.97us)         2
   nlp_jac_g  |  46.00us ( 15.33us)  46.27us ( 15.42us)         3
       total  |  11.23ms ( 11.23ms)   3.68ms (  3.68ms)         1
----------- MPC execution -----------
Predicted position of cube [11.232109659489, -3.0755623512201, 0.084989738824062]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 160.00us ( 53.33us) 161.27us ( 53.76us)         3
       nlp_g  | 461.00us (153.67us)  35.71us ( 11.90us)         3
    nlp_grad  | 188.00us (188.00us

----------- MPC execution -----------
Predicted position of cube [10.89355821408, -2.9759858200188, 0.085284686778788]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 231.00us ( 77.00us) 230.22us ( 76.74us)         3
       nlp_g  |  50.00us ( 16.67us)  50.12us ( 16.71us)         3
    nlp_grad  | 266.00us (266.00us) 266.39us (266.39us)         1
  nlp_grad_f  |   8.26ms (  2.75ms) 648.75us (216.25us)         3
  nlp_hess_l  |  10.57ms (  5.28ms)   2.57ms (  1.28ms)         2
   nlp_jac_g  |  76.00us ( 25.33us)  77.39us ( 25.80us)         3
       total  |  23.02ms ( 23.02ms)   6.10ms (  6.10ms)         1
----------- MPC execution -----------
Predicted position of cube [10.886931880026, -2.9739674488152, 0.085091961257441]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 155.00us ( 51.67us) 155.06us ( 51.69us)         3
       nlp_g  |  36.00us ( 12.00us)  35.49us ( 11.83us)         3
    nlp_grad  | 164.00us (164.00us)

----------- MPC execution -----------
Predicted position of cube [10.937986128436, -3.389646599935, 0.087698781147567]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 314.00us (104.67us) 189.81us ( 63.27us)         3
       nlp_g  |  43.00us ( 14.33us)  42.37us ( 14.12us)         3
    nlp_grad  | 212.00us (212.00us) 212.92us (212.92us)         1
  nlp_grad_f  | 629.00us (209.67us) 527.37us (175.79us)         3
  nlp_hess_l  |   7.76ms (  3.88ms)   2.22ms (  1.11ms)         2
   nlp_jac_g  |   6.58ms (  2.19ms)  84.51us ( 28.17us)         3
       total  |  62.52ms ( 62.52ms)   5.26ms (  5.26ms)         1
----------- MPC execution -----------
Predicted position of cube [10.940053319433, -3.4016374155757, 0.085930694858812]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 200.00us ( 66.67us) 200.19us ( 66.73us)         3
       nlp_g  |  45.00us ( 15.00us)  44.28us ( 14.76us)         3
    nlp_grad  | 213.00us (213.00us)

In [23]:
obj.end_simulation()

max(MPC_component.solver_times)

Ending simulation
numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
destroy semaphore
semaphore destroyed
Thread with taskId 0 exiting
Thread TERMINATED
destroy main semaphore
main semaphore destroyed
